In [1]:
# import pandas as pd
# anno_1 = 'D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\humananno_res\hanhui_oc.xlsx'
# anno_2 = 'D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\humananno_res\lsy_oc.xlsx'

# anno_1oc =pd.read_excel(anno_1, usecols=['Model','overall_consistency'])
# anno_2oc =pd.read_excel(anno_2, usecols=['Model','overall_consistency'])
# anno_1oc = anno_1oc.astype(object).where(pd.notnull(anno_1oc), None)
# anno_2oc = anno_2oc.astype(object).where(pd.notnull(anno_2oc), None)

In [2]:
import numpy as np

def rank_numbers(numbers):
    sorted_indices = sorted(range(len(numbers)), key=lambda k: numbers[k], reverse=True)
    ranks = [0] * len(numbers)

    i = 0
    while i < len(sorted_indices):
        value_indices = [i]
        while i + 1 < len(sorted_indices) and numbers[sorted_indices[i]] == numbers[sorted_indices[i + 1]]:
            i += 1
            value_indices.append(i)
        average_rank = np.mean([index + 1 for index in value_indices])
        for index in value_indices:
            ranks[sorted_indices[index]] = average_rank
        i += 1

    return ranks

def calculate_spearman_manual(values1, values2):
    n = len(values1)
    m = len(values2)
    rank1 = rank_numbers(values1)
    rank2 = rank_numbers(values2)
    d = np.array(rank1) - np.array(rank2)
    d_squared = np.square(d)
    spearman_corr = 1 - (6 * np.sum(d_squared)) / (n * (n**2 - 1))
    return spearman_corr

In [3]:
import json
jsonpath = 'D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\overall_consistency.json'
with open(jsonpath,'r') as f:
    oc = json.load(f)

history = r"D:\Astudying\VideoEval\HighlyHumanLikeVEB\GPT4o_eval_results\Old_overall_consistency_gpt4eval_results.json"
with open(history,'r') as f:
    gpt4o_eval_history = json.load(f)

In [4]:
# gptvsanno1_spearman = 0
# gptvsanno2_spearman = 0
# anno1vsanno2_spearman = 0
# gptvsannomean = 0
# badeval = []

# for i in range(len(oc)):
#     if i % 3 == 0:
#         gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 = human_anno[:,0]
#         anno2 = human_anno[:,1]
#         annomean = (anno1 + anno2)/2
#     else:
#         gpt4o_eval_rs+= np.array(list(oc[i]['gpt4o_eval'].values()))
#         human_anno = np.array(list(oc[i]['human_anno'].values()))
#         anno1 += human_anno[:,0]
#         anno2 += human_anno[:,1]
#         annomean += (anno1 + anno2)/2

#     if i % 3 == 2:
#         gptvsanno1_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno1)
#         gptvsanno2_spearman += calculate_spearman_manual(gpt4o_eval_rs,anno2)
#         anno1vsanno2_spearman += calculate_spearman_manual(anno1,anno2)
#         gptvsannomean += calculate_spearman_manual(gpt4o_eval_rs,annomean)

# gptvsanno1_spearman = 3*gptvsanno1_spearman/len(oc)
# gptvsanno2_spearman = 3*gptvsanno2_spearman/len(oc)
# anno1vsanno2_spearman = 3*  anno1vsanno2_spearman/len(oc)
# gptvsannomean = 3*gptvsannomean/len(oc)
# print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman)
# print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman)
# print("Anno1 vs Anno2 Spearman: ",anno1vsanno2_spearman)
# print("GPT vs AnnoMean Spearman: ",gptvsannomean)

In [5]:
gptvsanno1_spearman = np.zeros(len(oc))
gptvsanno2_spearman = np.zeros(len(oc))
anno1vsanno2_spearman = np.zeros(len(oc))
gptvsannomean_spearman = np.zeros(len(oc))
gptscore = np.zeros(np.array(list(oc[0]['gpt4o_eval'].values())).shape)
anno1score = np.zeros(np.array(list(oc[0]['gpt4o_eval'].values())).shape)
anno2score = np.zeros(np.array(list(oc[0]['gpt4o_eval'].values())).shape)
annomeanscore = np.zeros(np.array(list(oc[0]['gpt4o_eval'].values())).shape)

badeval = []

for i in range(len(oc)):
    gpt4o_eval_rs = np.array(list(oc[i]['gpt4o_eval'].values()))
    human_anno = np.array(list(oc[i]['human_anno'].values()))
    
    anno1 = human_anno[:,0]
    anno2 = human_anno[:,1]
    annomean = (anno1 + anno2)/2

    gptscore += gpt4o_eval_rs
    anno1score += anno1
    anno2score += anno2
    annomeanscore += annomean


    gptvsanno1 = calculate_spearman_manual(gpt4o_eval_rs,anno1)
    gptvsanno2 = calculate_spearman_manual(gpt4o_eval_rs,anno2)
    anno1vsanno2 = calculate_spearman_manual(anno1,anno2)
    gptvsannomean = calculate_spearman_manual(gpt4o_eval_rs,annomean)
    
    gptvsanno1_spearman[i] = gptvsanno1
    gptvsanno2_spearman[i] = gptvsanno2
    anno1vsanno2_spearman[i] = anno1vsanno2
    gptvsannomean_spearman[i] = gptvsannomean

    if gptvsannomean <0.3:
        badeval.append(i)

gptscore = gptscore/len(oc)
anno1score = anno1score/len(oc)
anno2score = anno2score/len(oc)
annomeanscore = annomeanscore/len(oc)

print("GPT vs Anno1 Spearman: ",gptvsanno1_spearman.mean())
print("GPT vs Anno2 Spearman: ",gptvsanno2_spearman.mean())
print("Anno1 vs Anno2 Spearman: ",anno1vsanno2_spearman.mean())
print("GPT vs AnnoMean Spearman: ",gptvsannomean_spearman.mean())
print("GPT average score: ",gptscore)
print("Anno1 average score: ",anno1score)
print("Anno2 average score: ",anno2score)
print("AnnoMean average score: ",annomeanscore)

GPT vs Anno1 Spearman:  0.513677536231884
GPT vs Anno2 Spearman:  0.661141304347826
Anno1 vs Anno2 Spearman:  0.5942028985507246
GPT vs AnnoMean Spearman:  0.6347826086956522
GPT average score:  [3.80072464 2.7826087  3.55072464 3.62681159 3.72101449]
Anno1 average score:  [3.70289855 3.09782609 3.38405797 3.52898551 3.75362319]
Anno2 average score:  [3.79347826 2.97463768 3.27898551 3.47101449 3.9057971 ]
AnnoMean average score:  [3.74818841 3.03623188 3.33152174 3.5        3.82971014]


In [6]:
from collections import Counter
badeval_prompt = []
for i in badeval:
    # print("spearman",gptvsannomean_spearman[i])
# print("videos",oc[i]['videos']['gen2'])
    # print("prompt",oc[i]['prompt_en'])
    badeval_prompt.append(oc[i]['prompt_en'])
    # print("GPT score",oc[i]['gpt4o_eval'])
    # print("Anno score",oc[i]['human_anno'])
    # print("GPT reasons",gpt4o_eval_history[str(i)])

In [7]:
conuter = Counter(badeval_prompt)
repeated = {k:v for k,v in conuter.items() if v>1}

In [8]:
print(sum(repeated.values()))
print(repeated)
with open('D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\BadEval4OverallConsistency.json','w') as f:
    json.dump(repeated,f)

22
{'An animated painting of fluffy white clouds moving in sky.': 2, 'A bigfoot walking in the snowstorm.': 2, 'Busy freeway at night.': 2, 'Sewing machine, old sewing machine working.': 3, 'Few big purple plums rotating on the turntable. water drops appear on the skin during rotation. isolated on the white background. close-up. macro.': 3, "Cinematic shot of van gogh's selfie, van gogh style": 2, 'A confused panda in calculus class': 2, 'A cute happy corgi playing in park, sunset': 2, 'A panda playing on a swing set': 2, 'An oil painting of a couple in formal evening wear going home get caught in a heavy downpour with umbrellas': 2}


In [9]:
len(badeval)

41